In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc

In [ ]:
path = '../raw_data/'

In [ ]:
at = pd.read_csv(path+'active_totes_20201210.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-0912.csv',names = ["timestamp","Pick Station","Availability","Blue Tote Loss","Grey Tote Loss"])
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'Faults20_11-10_12.csv')

In [ ]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

In [ ]:
at = feat.pre_process_AT(at)

In [ ]:
av = feat.pre_process_av(av)

In [ ]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

In [ ]:
fa_floor = feat.floor_shift_time_fa(fa, shift=15)

In [ ]:
fa_agg = feat.faults_aggregate(fa_floor,fault_agg_level= 'Asset Code', agg_type = 'count')

In [ ]:
av_sel,at_sel = feat.av_at_select(av, at, remove_high_AT = True, AT_limit = None)

av_agg = feat.aggregate_availability(av_sel, agg_level = 'Quadrant')
at_agg = feat.aggregate_totes(at_sel, agg_level = 'Quadrant')

In [ ]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg , agg_level = 'Quadrant')

In [ ]:
df['timestamp'].dt.date.nunique()

In [ ]:
df['Availability'] = 1 - df['Downtime']
df = df.drop(['Downtime'], axis=1)
df = df[df['TOTES'] > 5]
df['log_totes'] = np.log(df['TOTES'])

In [ ]:
X,y = md.gen_feat_var(df,target = 'Availability', features = ['Totes','Faults'])

In [ ]:
X_run = X.drop(['log_totes', 'Availability'], axis=1)

In [ ]:
sns.lmplot(x='log_totes', y='Availability', data=df)
plt.xlim(0, 5)

In [ ]:
X_train, X_test, y_train, y_test = md.split(X_run,y)

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

cv_R2 = md.cross_validate_r2(Linear_mdl, X_run, y, n_folds = 5, shuffle = True, random_state = 101)

In [ ]:
# X_train, X_test, y_train, y_test = md.split(X[['TOTES']],y)

# Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

# cv_R2 = md.cross_validate_r2(Linear_mdl, X[['TOTES']], y, n_folds = 5, shuffle = True, random_state = 101)

In [ ]:
Linear_mdl.coef_

### Add LM - metrics

In [ ]:
X_train, X_test, y_train, y_test = md.split(X, y)

In [ ]:
X_train_rd = X_train
X = 10

def find_features(X_train, y_train, X):
    max_p = 1
    while max_p > 0.1:
        model = sm.OLS(y_train, X_train)
        results = model.fit()
        top_X = results.pvalues.sort_values(ascending=False).head(X)
        max_p = top_X.tail(X).values[0]
        rm_col = list(results.pvalues.sort_values(ascending=False).head(X).index)
        X_train = X_train.drop(rm_col, axis=1)
    return X_train.columns

In [ ]:
cols = find_features(X_train=X_train_rd, y_train=y_train, X=X)

In [ ]:
model = sm.OLS(y_train, X_train[cols])
results = model.fit()
print(results.summary())

1. Select negative Coeffs
2. Take "recent faults" - ////////////////////////////////////////////////////[///////]
4. PS Fault data cut
5. Merge with Coefficients (on=Asset Code)
6. Coefficient * Duration (or 1/0 occurrence)
7. Output: Lost time = (6) (at pick station from Asset code fault)
8. Append (4) together - feed into dashboard
- group by Asset Code, sum lost time

In [ ]:
filt_columns = X[keep_features]
X_train, X_test, y_train, y_test = md.split(X[keep_features],y)

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test,
                                                              fit_intercept=False)

cv_R2 = md.cross_validate_r2(Linear_mdl, X_train[keep_features], y)

# Hold out Test X_test -> y_test

#y_prediction =  results.predict(X_test)

# plt.scatter(y_prediction,y_test)

# from sklearn.metrics import r2_score
# print(r2_score(y_prediction,y_test))

In [ ]:
print(results.summary())

In [ ]:
fa_Both = fa[fa['Tote Colour']=="Both"]
fa_Both = fa_Both[fa_Both['Pick Station']==False]
fa_Both[fa_Both['Desk']!="Z"]['Asset Code'].value_counts()

In [ ]:
fa[fa['Fault ID']==7647]

In [ ]:
fa[fa['Pick Station']!=False]['Asset Code'].value_counts()

In [ ]:
X

In [ ]:
y

In [ ]:
at_agg.loc["2020-11-20":].head(50)